In [81]:
# Visualization configuration (tweakable knobs)
cfg = {
    # Data
    'year': 2014,
    'file_name': '../data/processed/simulations/simulations_apr7.parquet',

    # Layout sizing
    'target_width': 900,
    'margins': {'l': 110, 'r': 210, 't': 130, 'b': 90},
    'spacing': {'horizontal': 0.002, 'vertical': 0.002},
    'spacing_mode': 'auto-match',  # 'fixed' uses values above, 'auto-match' aligns subplot gaps to cell gaps

    # Heatmap look
    'colorscale': 'Viridis',
    'xgap': 0.5,
    'ygap': 0.5,

    # Colors and fonts
    'plot_bg_color': 'white',
    'paper_bg_color': 'white',
    'font_color': 'black',
    'base_font_size': 10,

    # Colorbar and labels positions
    'colorbar': {'x': 1.14, 'thickness': 15},
    'colorbar_label': {'text': 'Profit % Change', 'x': 1.23, 'angle': 90},
    'v6_axis_label': {
        'text': 'Proportion of Cultivars Susceptible to V6-virulent Strains',
        'x': 1.09,
        'angle': 90,
    },
    'percent_v6_numbers_x': 1.015,

    # Demand and percentile labels
    'demand_label': {'x': -0.07, 'angle': -90, 'font_size': 10},
    'percentile_label': {'x': -0.11, 'text': 'Percentile of Dispersal-Centrality', 'angle': -90, 'font_size': 12},

    # Column headers
    'p0_label': {'y': 1.05, 'font_size': 11, 'scientific': True, 'sig_figs': 1},
    'p0_title': {'y': 1.13, 'font_size': 12, 'text': 'Initial Probability of Disease'},

    # Shared x-axis title
    'x_title': {'y': -0.13, 'font_size': 12, 'text': 'Number of Sprays in May'},

    # Tick styles (outer axes only)
    'ticks': {
        'len': 3,
        'width': 1,
        'color': 'rgba(0,0,0,0.4)',
        'font_size': 8,
        'outer_only': True,
        'min_mid_max': True,
    },

    # Brackets near demand labels
    'brackets': {
        'x_inner': -0.058,
        'x_outer': -0.045,
        'gap': 0.01,
        'line_color': 'rgba(0,0,0,0.9)',
        'line_width': 1.2,
    },

    # Outside tick marker lines
    'outside_ticks': {
        'top_height': 0.012,
        'right_width': 0.008,
        'color': 'rgba(0,0,0,0.35)',
        'width': 1,
    },

    # Boxes for specific labels (bg and border for descriptive labels only)
    'box_style': {
        'bg': 'rgba(255,255,255,0.9)',
        'border_color': 'rgba(0,0,0,0.25)',
        'border_width': 0.6,
        'pad': 2,
    },

    # Indicator controls
    'indicator': {
        'enabled': True,           # toggle on/off
        'style': 'dot',            # one of: 'dot', 'ring', 'cross'
        'size': 4,                 # base size (smaller default)
        # base colors; some styles will override
        'fill': 'rgba(255,255,255,0.9)',
        'border_color': 'rgba(0,0,0,0.85)',
        'border_width': 0.6,
    },

    # Export options
    'export': {
        'html': False,
        'png': False,  # requires kaleido
        'png_scale': 2,
        'file_prefix': 'profit_heatmap_all_demands',
    }
}


In [86]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Pull configuration with defaults if cfg is not defined
try:
    _cfg = cfg
except NameError:
    _cfg = {}

# Define parameters
year = _cfg.get('year', 2014)  # 2014, 2015, 2016, 2017

# --- Step 1: Read the full data for the year to get global min/max ---
file_name = _cfg.get('file_name', '../data/processed/simulations/simulations_apr7.parquet')
df_full_year = pd.read_parquet(file_name)
df_full_year = df_full_year[df_full_year['Year'] == year]

# --- Step 2: Determine global color scale limits for the entire year ---
v_min = df_full_year['Mean Profit Percent Change'].min()
v_max = df_full_year['Mean Profit Percent Change'].max()
print(f"Global Min/Max for {year}: {v_min:.2f}%, {v_max:.2f}%")

# --- Step 3: Keep ALL market demand levels (no filtering) ---
df_profit_all = df_full_year.copy()
# Normalize demand names to lowercase for consistent filtering
md_lower = df_profit_all['Market Demand'].astype(str).str.lower()
df_profit_all = df_profit_all.assign(_md_lower=md_lower)

# Define demand level order (top to bottom). Only include those present in the data.
preferred_order = ['high', 'moderate', 'low']
available = list(sorted(df_profit_all['_md_lower'].unique()))
demand_levels = [d for d in preferred_order if d in available] + [d for d in available if d not in preferred_order]

# Get sorted unique V6 Percent and Initial Probability values across all demands
unique_v6_percent = sorted(df_profit_all['V6 Percent'].unique())
unique_init_prob = sorted(df_profit_all['Initial Probability'].unique())

# Standardize categories for all subplots
sprays_all = sorted(df_profit_all['Sprays in May'].unique())
quantiles_all = sorted(df_profit_all['Quantile'].unique())

# Format initial probabilities: base-10 scientific notation (e.g., 5×10⁻⁵)
_p0_cfg = _cfg.get('p0_label', {'y': 1.05, 'font_size': 11, 'scientific': True, 'sig_figs': 1})
use_sci = bool(_p0_cfg.get('scientific', True))
sig_figs = int(_p0_cfg.get('sig_figs', 1))

if use_sci:
    supers = str.maketrans({'-':'⁻','0':'⁰','1':'¹','2':'²','3':'³','4':'⁴','5':'⁵','6':'⁶','7':'⁷','8':'⁸','9':'⁹'})
    def fmt_prob(prob: float) -> str:
        if prob == 0:
            return '0'
        exp = int(np.floor(np.log10(abs(prob))))
        mant = prob / (10**exp)
        mant_str = f"{mant:.{max(0, sig_figs-1)}f}".rstrip('0').rstrip('.')
        exp_str = str(exp).translate(supers)
        return f"{mant_str}×10{exp_str}"
else:
    def fmt_prob(prob: float, decimal_places: int = 5) -> str:
        formatted = f"{prob:.{decimal_places}f}".rstrip('0').rstrip('.')
        return formatted if formatted else '0'

unique_init_prob_labels = [fmt_prob(prob) for prob in unique_init_prob]

# Define the grid size (stack each demand's V6 rows vertically)
v6_rows = len(unique_v6_percent)
n_rows = len(demand_levels) * v6_rows
n_cols = len(unique_init_prob)

# Margins and spacing
_margins = _cfg.get('margins', {'l': 110, 'r': 210, 't': 130, 'b': 90})
left_margin = _margins.get('l', 110)
right_margin = _margins.get('r', 210)
top_margin = _margins.get('t', 130)
bottom_margin = _margins.get('b', 90)

_spacing = _cfg.get('spacing', {'horizontal': 0.002, 'vertical': 0.002})
spacing_mode = _cfg.get('spacing_mode', 'auto-match')

xgap = float(_cfg.get('xgap', 0.5))
ygap = float(_cfg.get('ygap', 0.5))

# Compute figure width/height for square cells using standardized categories
example_nx = len(sprays_all)
example_ny = len(quantiles_all)
target_width = int(_cfg.get('target_width', 900))
avail_width = target_width - left_margin - right_margin
height_available = avail_width * (n_rows * example_ny) / (n_cols * example_nx)
target_height = int(height_available + top_margin + bottom_margin)

# Subplot spacing: clamp to tiny values to minimize inter-panel whitespace
if spacing_mode == 'auto-match' and example_nx > 1 and example_ny > 1:
    horizontal_spacing = float(max(0.0002, min(0.004, (xgap / max(1, example_nx-1)) * 0.01)))
    vertical_spacing   = float(max(0.0002, min(0.004, (ygap / max(1, example_ny-1)) * 0.01)))
else:
    horizontal_spacing = float(_spacing.get('horizontal', 0.002))
    vertical_spacing = float(_spacing.get('vertical', 0.002))

# Equal domain allocation for all mini heatmaps
row_heights = [1.0 / n_rows] * n_rows
column_widths = [1.0 / n_cols] * n_cols

# Create subplot titles as empty strings
subplot_titles = [''] * (n_rows * n_cols)

# Create the figure with subplots (explicit domain sizes; categorical axes later)
fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=subplot_titles,
    horizontal_spacing=horizontal_spacing,
    vertical_spacing=vertical_spacing,
    shared_xaxes=False,
    shared_yaxes=False,
    row_heights=row_heights,
    column_widths=column_widths,
    specs=[[{'type': 'xy'} for _ in range(n_cols)] for _ in range(n_rows)]
)

# Colorscale
colorscale_choice = _cfg.get('colorscale', 'Viridis')

# Indicator controls and styling
_indicator_cfg = _cfg.get('indicator', {})
ind_enabled = bool(_indicator_cfg.get('enabled', True))
ind_style = str(_indicator_cfg.get('style', 'dot')).lower()
ind_size = int(_indicator_cfg.get('size', 4))
ind_fill = _indicator_cfg.get('fill', 'rgba(255,255,255,0.9)')
ind_border_color = _indicator_cfg.get('border_color', 'rgba(0,0,0,0.85)')
ind_border_width = _indicator_cfg.get('border_width', 0.6)

style_map = {
    'dot':    dict(symbol='circle', size=ind_size, color=ind_fill, line=dict(color=ind_border_color, width=ind_border_width)),
    'ring':   dict(symbol='circle-open', size=max(3, ind_size+1), color=ind_border_color, line=dict(color=ind_border_color, width=ind_border_width+0.2)),
    'cross':  dict(symbol='x', size=max(4, ind_size+2), color=ind_border_color, line=dict(color=ind_border_color, width=ind_border_width+0.2)),
}
marker_style = style_map.get(ind_style, style_map['dot'])

# Build category label lists (strings) for categorical axes
x_categories = [str(s) for s in sprays_all]
y_categories = [('{:.2f}'.format(q)).rstrip('0').rstrip('.') for q in quantiles_all]

# Add heatmaps for each panel, filling missing with all-NaN arrays to preserve equal shapes
for d_idx, demand in enumerate(demand_levels):
    for v_idx, v6_value in enumerate(unique_v6_percent):
        base_row = d_idx * v6_rows + v_idx + 1
        subset_dv6 = df_profit_all[(df_profit_all['_md_lower'] == demand) & (df_profit_all['V6 Percent'] == v6_value)]
        for col_idx, init_prob_value in enumerate(unique_init_prob, start=1):
            df_filtered = subset_dv6[subset_dv6['Initial Probability'] == init_prob_value]
            if df_filtered.empty:
                z_vals = np.full((len(y_categories), len(x_categories)), np.nan)
            else:
                pivot_profit = df_filtered.pivot_table(
                    values='Mean Profit Percent Change',
                    index='Quantile', columns='Sprays in May', aggfunc='mean'
                )
                pivot_profit = pivot_profit.reindex(index=quantiles_all, columns=sprays_all)
                z_vals = pivot_profit.values
            fig.add_trace(
                go.Heatmap(
                    z=z_vals,
                    x=x_categories,
                    y=y_categories,
                    colorscale=colorscale_choice,
                    zmin=v_min, zmax=v_max,
                    coloraxis='coloraxis',
                    showscale=False,
                    xgap=_cfg.get('xgap', 0.5),
                    ygap=_cfg.get('ygap', 0.5),
                    hoverongaps=False
                ),
                row=base_row, col=col_idx
            )
            if ind_enabled and not df_filtered.empty:
                try:
                    z = z_vals
                    if np.all(np.isnan(z)):
                        pass
                    else:
                        max_val = np.nanmax(z)
                        locs = np.where(z == max_val)
                        if len(locs[0]) > 0:
                            r_idx = int(locs[0][0])
                            c_idx = int(locs[1][0])
                            x_best = x_categories[c_idx]
                            y_best = y_categories[r_idx]
                            fig.add_trace(
                                go.Scatter(x=[x_best], y=[y_best], mode='markers', marker=marker_style, hoverinfo='skip', showlegend=False),
                                row=base_row, col=col_idx
                            )
                except Exception:
                    pass

# Layout, colorbar, and axes constraints
plot_bg_color = _cfg.get('plot_bg_color', 'white')
paper_bg_color = _cfg.get('paper_bg_color', 'white')
font_color = _cfg.get('font_color', 'black')
base_font_size = _cfg.get('base_font_size', 10)

fig.update_layout(
    template='plotly_white', plot_bgcolor=plot_bg_color, paper_bgcolor=paper_bg_color,
    font=dict(color=font_color, size=base_font_size),
    title=dict(text=f'Relative Change in Profit across Market Demand Levels in {year}', font=dict(color=font_color, size=18), x=0.5, y=0.98, xanchor='center', yanchor='top'),
    margin=dict(l=left_margin, r=right_margin, t=top_margin, b=bottom_margin),
    coloraxis=dict(colorscale=colorscale_choice, cmin=v_min, cmax=v_max, colorbar=dict(title=dict(text='', side='right', font=dict(color=font_color)), tickfont=dict(size=10, color=font_color), len=1.0, y=0.5, x=_cfg.get('colorbar', {}).get('x', 1.14), thickness=_cfg.get('colorbar', {}).get('thickness', 15))),
    shapes=[]
)

# Categorical axes and constraints
fig.update_xaxes(type='category', categoryorder='array', categoryarray=x_categories, showgrid=False, automargin=False, constrain='domain')
fig.update_yaxes(type='category', categoryorder='array', categoryarray=y_categories, showgrid=False, automargin=False, constrain='domain')

# Remove tick marks/axis lines initially (we'll re-enable on outer edges)
fig.update_xaxes(ticks="", ticklen=0, tickwidth=0, tickcolor="rgba(0,0,0,0)", showline=False, zeroline=False, mirror=False)
fig.update_yaxes(ticks="", ticklen=0, tickwidth=0, tickcolor="rgba(0,0,0,0)", showline=False, zeroline=False, mirror=False)

# Left labels and other annotations
_demand_label = _cfg.get('demand_label', {'x': -0.07, 'angle': -90, 'font_size': 10})
for d_idx, demand in enumerate(demand_levels):
    y_center = 1 - (((d_idx * v6_rows) + (v6_rows / 2)) / n_rows)
    fig.add_annotation(dict(text=demand.capitalize(), x=_demand_label.get('x', -0.07), y=y_center, xref='paper', yref='paper', textangle=_demand_label.get('angle', -90), showarrow=False, font=dict(size=_demand_label.get('font_size', 10), color=font_color), align='center', xanchor='right', yanchor='middle'))

_percentile_label = _cfg.get('percentile_label', {'x': -0.11, 'text': 'Percentile of Dispersal-Centrality', 'angle': -90, 'font_size': 12})
_box_style = _cfg.get('box_style', {'bg': 'rgba(255,255,255,0.9)', 'border_color': 'rgba(0,0,0,0.25)', 'border_width': 0.6, 'pad': 2})
fig.add_annotation(dict(text=_percentile_label.get('text', 'Percentile of Dispersal-Centrality'), x=_percentile_label.get('x', -0.11), y=0.5, textangle=_percentile_label.get('angle', -90), xref='paper', yref='paper', showarrow=False, font=dict(size=_percentile_label.get('font_size', 12), color=font_color), align='center', xanchor='right', yanchor='middle', bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'), bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'), borderwidth=_box_style.get('border_width', 0.6), borderpad=_box_style.get('pad', 2)))

# Column headers (p0 values) using base-10 sci notation
for idx, init_prob_label in enumerate(unique_init_prob_labels):
    x_pos = (idx + 1 - 0.5) / n_cols
    fig.add_annotation(dict(text=f"{init_prob_label}", x=x_pos, y=_p0_cfg.get('y', 1.05), xref='paper', yref='paper', showarrow=False, font=dict(size=_p0_cfg.get('font_size', 11), color=font_color), align='center', xanchor='center'))

_p0_title = _cfg.get('p0_title', {'y': 1.13, 'font_size': 12, 'text': 'Initial Probability of Disease'})
fig.add_annotation(dict(text=_p0_title.get('text', 'Initial Probability of Disease'), x=0.5, y=_p0_title.get('y', 1.13), xref='paper', yref='paper', showarrow=False, font=dict(size=_p0_title.get('font_size', 12), color=font_color), align='center', xanchor='center', bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'), bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'), borderwidth=_box_style.get('border_width', 0.6), borderpad=_box_style.get('pad', 2)))

# Right numeric V6 labels and right vertical label
percent_v6_x_pos = _cfg.get('percent_v6_numbers_x', 1.015)
for d_idx, _d in enumerate(demand_levels):
    for v_idx, v6_value in enumerate(unique_v6_percent):
        row_index = d_idx * v6_rows + v_idx
        y_pos = 1 - ((row_index + 0.5) / n_rows)
        prop_label = ('{:.2f}'.format(float(v6_value))).rstrip('0').rstrip('.')
        fig.add_annotation(dict(text=prop_label, x=percent_v6_x_pos, y=y_pos, xref='paper', yref='paper', showarrow=False, font=dict(size=11, color=font_color), align='right', xanchor='left', yanchor='middle'))

_v6_axis_label = _cfg.get('v6_axis_label', {'text': 'Proportion of Cultivars Susceptible to V6-virulent Strains', 'x': 1.09, 'angle': 90})
fig.add_annotation(dict(text=_v6_axis_label.get('text', 'Proportion of Cultivars Susceptible to V6-virulent Strains'), x=_v6_axis_label.get('x', 1.09), y=0.5, xref='paper', yref='paper', textangle=_v6_axis_label.get('angle', 90), showarrow=False, font=dict(size=12, color=font_color), align='center', xanchor='left', yanchor='middle', bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'), bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'), borderwidth=_box_style.get('border_width', 0.6), borderpad=_box_style.get('pad', 2)))

_cbar_label = _cfg.get('colorbar_label', {'text': 'Profit % Change', 'x': 1.23, 'angle': 90})
fig.add_annotation(dict(text=_cbar_label.get('text', 'Profit % Change'), x=_cbar_label.get('x', 1.23), y=0.5, xref='paper', yref='paper', textangle=_cbar_label.get('angle', 90), showarrow=False, font=dict(size=12, color=font_color), align='center', xanchor='left', yanchor='middle'))

# Brackets and border
shapes = []
_br = _cfg.get('brackets', {'x_inner': -0.058, 'x_outer': -0.045, 'gap': 0.01, 'line_color': 'rgba(0,0,0,0.9)', 'line_width': 1.2})
x_inner = _br.get('x_inner', -0.058)
x_outer = _br.get('x_outer', -0.045)
line_style = dict(color=_br.get('line_color', 'rgba(0,0,0,0.9)'), width=_br.get('line_width', 1.2))
gap = _br.get('gap', 0.01)
for d_idx in range(len(demand_levels)):
    y_top = 1 - (d_idx * v6_rows) / n_rows
    y_bottom = 1 - ((d_idx + 1) * v6_rows) / n_rows
    if d_idx == 0:
        y0 = y_bottom + gap
        y1 = y_top
    elif d_idx == len(demand_levels) - 1:
        y0 = y_bottom
        y1 = y_top - gap
    else:
        y0 = y_bottom + gap
        y1 = y_top - gap
    shapes.append(dict(type='line', xref='paper', yref='paper', x0=x_inner, x1=x_inner, y0=y0, y1=y1, line=line_style))
    shapes.append(dict(type='line', xref='paper', yref='paper', x0=x_inner, x1=x_outer, y0=y1, y1=y1, line=line_style))
    shapes.append(dict(type='line', xref='paper', yref='paper', x0=x_inner, x1=x_outer, y0=y0, y1=y0, line=line_style))
shapes.append(dict(type='rect', xref='paper', yref='paper', x0=0, y0=0, x1=1, y1=1, line=dict(color='rgba(0,0,0,0.15)', width=1)))
fig.update_layout(shapes=shapes)

# Outer-axis ticks only on the leftmost column and bottom row using categorical tick labels
_ticks = _cfg.get('ticks', {'len': 3, 'width': 1, 'color': 'rgba(0,0,0,0.4)', 'font_size': 8, 'outer_only': True, 'min_mid_max': True})
fig.update_xaxes(tickmode='array', tickvals=x_categories, ticktext=x_categories, ticks='outside', ticklen=_ticks.get('len', 3), tickwidth=_ticks.get('width', 1), tickcolor=_ticks.get('color', 'rgba(0,0,0,0.4)'), tickfont=dict(size=_ticks.get('font_size', 8), color=font_color))
fig.update_yaxes(tickmode='array', tickvals=y_categories, ticktext=y_categories, ticks='outside', ticklen=_ticks.get('len', 3), tickwidth=_ticks.get('width', 1), tickcolor=_ticks.get('color', 'rgba(0,0,0,0.4)'), tickfont=dict(size=_ticks.get('font_size', 8), color=font_color))

for r in range(1, n_rows + 1):
    for c in range(1, n_cols + 1):
        show_y = (c == 1)
        show_x = (r == n_rows)
        fig.update_yaxes(row=r, col=c, showticklabels=show_y, ticks=('outside' if show_y else ''), ticklen=(_ticks.get('len', 3) if show_y else 0), tickwidth=(_ticks.get('width', 1) if show_y else 0))
        fig.update_xaxes(row=r, col=c, showticklabels=show_x, ticks=('outside' if show_x else ''), ticklen=(_ticks.get('len', 3) if show_x else 0), tickwidth=(_ticks.get('width', 1) if show_x else 0))

# Shared x title
_x_title = _cfg.get('x_title', {'y': -0.13, 'font_size': 12, 'text': 'Number of Sprays in May'})
fig.add_annotation(dict(text=_x_title.get('text', 'Number of Sprays in May'), x=0.5, y=_x_title.get('y', -0.13), xref='paper', yref='paper', showarrow=False, font=dict(size=_x_title.get('font_size', 12), color=font_color), align='center', bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'), bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'), borderwidth=_box_style.get('border_width', 0.6), borderpad=_box_style.get('pad', 2)))

# Final figure size
fig.update_layout(width=target_width, height=target_height)

# Hover template
fig.update_traces(hovertemplate='Sprays in May: %{x}<br>Dispersal Percentile: %{y}<br>Profit % Change: %{z:.2f}%<extra></extra>')

fig.show()

_export = _cfg.get('export', {'html': False, 'png': False, 'png_scale': 2, 'file_prefix': 'profit_heatmap_all_demands'})
_fname_base = f"{_export.get('file_prefix', 'profit_heatmap_all_demands')}_{year}_white_{colorscale_choice}_global_scale"
if _export.get('html', False):
    fig.write_html(f'{_fname_base}.html')
if _export.get('png', False):
    fig.write_image(f'{_fname_base}.png', scale=int(_export.get('png_scale', 2)))


Global Min/Max for 2014: -11.34%, 0.37%
